In [12]:
import os
from pathlib import Path
from fdua_competition.vectorstore import build_vectorstore, get_document_list, get_documents_dir
from fdua_competition.enums import Mode, VectorStoreOption, ChatModelOption
from fdua_competition.rag import get_chat_model, read_prompt
from langchain_core.prompts import ChatPromptTemplate
from fdua_competition.utils import get_queries
from tqdm import tqdm
from pydantic import BaseModel, Field

In [2]:
query = "extract name of organization"

In [3]:
db = build_vectorstore(os.getenv("OUTPUT_NAME"), Mode.TEST, VectorStoreOption.CHROMA)

[prepare_vectorstore] chroma: /fdua-competition/.fdua-competition/vectorstores/chroma


In [4]:
class Organizations(BaseModel):
    organizations: list[str] = Field(description="[company names]")
    source: str = Field(description="source file")


def extract_organization_names(path: Path) -> Organizations:
    query = "extract all the company names mentioned in the documents"
    prompt_template = ChatPromptTemplate.from_messages(
        [
            ("system", read_prompt("information_extractor")),
            ("system", "documents:\n{documents}"),
            ("user", "{query}")
        ]
    )
    documents = db.as_retriever().invoke(query, filter={"source": str(path)})
    chat_model = get_chat_model(ChatModelOption.AZURE).with_structured_output(Organizations)
    chain = prompt_template | chat_model
    return chain.invoke({"documents": documents, "query": query})

In [5]:
docs = get_document_list(get_documents_dir(Mode.TEST))
orgs = [extract_organization_names(doc) for doc in tqdm(docs, desc="extrcting organization names...")]

extrcting organization names...: 100% 10/10 [00:49<00:00,  4.99s/it]


In [6]:
for org in orgs:
    print(org.source)
    print(org.organizations)
    print()

/fdua-competition/.fdua-competition/validation/documents/9.pdf
['東洋紡', 'TOYOBO', '東洋紡グループ']

/fdua-competition/.fdua-competition/validation/documents/8.pdf
['dentsu', '株式会社電通グループ']

/fdua-competition/.fdua-competition/validation/documents/10.pdf
['日本化薬グループ', 'ニッポンカヤクコリア', 'ポラテクノ', 'ニッポンカヤク（タイランド）', '台湾日化股', 'モクステック', 'ニッポンカヤクアメリカ', 'カヤク アドバンスト マテリアルズ', 'カヤク セイフティシステムズ デ メキシコ', '厚和産業', 'ニッカファインテクノ', 'テイコクテーピングシステム', '上海化耀国際貿易', '無錫先進化薬化', '化薬化工（無錫）', '無錫宝来光学科技', 'レイスペック', 'デジマ オプティカル フィルムズ', 'ユーロニッポンカヤク', 'カヤク セイフティシステムズ ヨーロッパ', '日本化薬フードテク', '和光都市開発', '化薬（湖州）安全器材', '化薬（上海）管理', 'カヤク セイフティシステムズ マレーシア', '化薬（湖州）安全器材有限公司', 'カヤク セイフティシステムズ ヨーロッパ a.s.', 'カヤク セイフティシステムズ デ メキシコ, S.A. de C.V.', 'カヤク セイフティシステムズ マレーシア Sdn.Bhd.', 'ニッポンカヤクアメリカ, INC.', '株式会社ポラテクノ', 'カヤク アドバンスト マテリアルズ, Inc.', 'テイコクテーピングシステム株式会社', '無錫先進化薬化工有限公司', '上海化耀国際貿易有限公司', 'ニッポンカヤク（タイランド） CO., LTD.', 'モクステック,Inc.', '無錫宝来光学科技有限公司', 'デジマ オプティカル フィルムズ B.V.', 'レイスペック Ltd.', '株式会社ニッカファインテクノ', 'ニッポンカヤクコリア.Co.,Ltd', 'ユーロニッポンカヤク GmbH']

/

In [7]:
class SourceToLookup(BaseModel):
    query: str = Field(description="query")
    source: str = Field(description="source file to look up")


def search_source_to_lookup(query, orgs):    
    prompt_template = ChatPromptTemplate.from_messages(
        [
            ("system", read_prompt("retrieval_assistant")),
            ("system", "organizations:\n{organization}"),
            ("user", "query: {query}")
        ]
    )
    chat_model = get_chat_model(ChatModelOption.AZURE).with_structured_output(SourceToLookup)
    chain = prompt_template | chat_model
    context = "\n\n".join([f"- {org.source}: {org.organizations}" for org in orgs])
    return chain.invoke({"organization": context, "query": query})

In [8]:
docs_to_lookup = [search_source_to_lookup(query, orgs) for query in tqdm(get_queries(Mode.TEST), desc="finding source to reference...")]

finding source to reference...: 100% 50/50 [01:49<00:00,  2.19s/it]


In [9]:
docs_to_lookup

[SourceToLookup(query='大成温調が積極的に資源配分を行うとしている高付加価値セグメントを全てあげてください。', source='/fdua-competition/.fdua-competition/validation/documents/6.pdf'),
 SourceToLookup(query='花王の生産拠点数は何拠点ですか？', source='/fdua-competition/.fdua-competition/validation/documents/4.pdf'),
 SourceToLookup(query='電通グループPurposeは何ですか？', source='/fdua-competition/.fdua-competition/validation/documents/8.pdf'),
 SourceToLookup(query='2023年度の大成温調の連結純資産配当率（DOE）は何%でしたか？', source='/fdua-competition/.fdua-competition/validation/documents/6.pdf'),
 SourceToLookup(query='ダイドーグループの従業員数において、2012年から2023年までの12年間で、医薬品関連が食品を下回った年を全てあげてください。', source='/fdua-competition/.fdua-competition/validation/documents/3.pdf'),
 SourceToLookup(query='東洋紡の取締役の在籍期間において、0~3年と4~9年ではどちらの方が取締役の人数が多いか', source='/fdua-competition/.fdua-competition/validation/documents/9.pdf'),
 SourceToLookup(query='東洋紡グループのコア技術を4つ答えてください。', source='/fdua-competition/.fdua-competition/validation/documents/9.pdf'),
 SourceToLookup(query='ダイドーグループが2012年に立ち上げたチャネルの国内飲料事業の中での売

In [19]:
class ResearchAssistantResponse(BaseModel):
    query: str = Field(description="the query that was asked.")
    response: str = Field(description="the answer for the given query")
    reason: str = Field(description="the reason for the response.")
    organization_name: str = Field(description="the organization name that the query is about.")
    contexts: list[str] = Field(description="the context that the response was based on with its file path and page number.")


def answer_query(query: str, reference: str) -> Organizations:
    prompt_template = ChatPromptTemplate.from_messages(
        [
            ("system", read_prompt("research_assistant")),
            ("system", "context:\n{context}"),
            ("user", query)
        ]
    )
    context = db.as_retriever().invoke(query, filter={"source": reference})
    chat_model = get_chat_model(ChatModelOption.AZURE).with_structured_output(ResearchAssistantResponse)
    chain = prompt_template | chat_model
    return chain.invoke({"context": context, "language": "japanese"})


for i in docs_to_lookup:
    print(answer_query(i.query, i.source))

KeyError: 'Input to ChatPromptTemplate is missing variables {\'\\n  "query"\'}.  Expected: [\'\\n  "query"\', \'context\', \'language\'] Received: [\'context\', \'language\']\nNote: if you intended {\n  "query"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{\n  "query"}}\'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT '